# Úkol č. 1 - vizualizace dat a web scraping (do 20. října)

  * V rámci tohoto úkolu musíte stáhnout dat z webu (tzv. _web scraping_, velmi základní) a následně data zpracovat a vizualizovat.
  * Cílem bude stáhnout data ze serveru https://www.volby.cz týkající se voleb do zastupitelstva Vámi vybraného (většího) města, uložit tato data v tabulkovém formátu a pak vymyslet vizualizace a zobrazení dat, které umožní orientaci v těchto datech a zvýrazní zajímavé informace a zobrazit přehledně časový vývoj různých veličin.
 
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Výběr zdroje dat

Vyberte si větší město, které má zastupitelstvo druhu 3 (Zastupitelstvo statutárního města) a strojově stáhněte informace o stranách a kandidátkách z následujících let:
 * [2002](https://www.volby.cz/pls/kv2002/kv12?xjazyk=CZ&xid=0), [2006](https://www.volby.cz/pls/kv2006/kv12?xjazyk=CZ&xid=0), [2010](https://www.volby.cz/pls/kv2010/kv12?xjazyk=CZ&xid=0), [2014](https://www.volby.cz/pls/kv2014/kv12?xjazyk=CZ&xid=0) a [2018](https://www.volby.cz/pls/kv2018/kv12?xjazyk=CZ&xid=0).
 

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * Strojově stáhněte data pro vybrané město a uložte je všechny do (asi dvou) přehledných tabulek ve formátu _csv_.
  * Data musí obsahovat _alespoň_ toto:
    * Vývoj výsledků (v procentech i počtu hlasů) pro jednotlivé strany v jednotlivých letech.
    * Seznam všech kandidátů všech stran v jednotlivých letech, u kandidáta by mělo být zaznamenáno: jméno, věk v době voleb, navrhující strana, politická příslušnost, volební zisk (procento i počet hlasů), pořadí na kandidátce, pořadí zvolení, jestli získal mandát (tyto informace získáte souhrnně ve _jmenných seznamech_).
  * V druhé části Vašeho Jupyter notebooku pracujte s těmito tabulkami načtenými z _csv_ souboru (aby opravující nemusel spouštět stahování z webu).
  * Tabulky ve formátu _csv_ také odevzdejte.
  * S využitím vybraných nástrojů zpracujte data a vymyslete vizualizace a grafy, aby bylo vidět následující:
    * Časový vývoj (po rocích voleb) počtu kandidujících stran i lidí a to celkově i po jednotlivých stranách (ve volbách, kterých se daná strana účastnila).
    * Věkovou strukturu kandidátů celkově i za jednotlivé strany a vývoj této struktury během jednotlivých voleb.
    * Časový vývoj volební účasti a volebních výsledků jednotlivých stran.
    * Časový vývoj podílu kandidujících s titulem a bez titulu.

**Další body zadání** za případné další body (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +2 body) U titulů se pokuste rozlišit i různé stupně vzdělání: bakalářský, magisterský, doktorský a vyšší, vojenská hodnost atp. Zkuste odhadnout i podíl žen na kandidátkách.
  * (až +4 body) Pokuste se u jednotlivých kandidátů zjistit, zda kandidovali ve více volbách. Najděte 10 nejpilnějších kandidátů a vypište jejich volební zisky a za jaké strany kandidovali.
  * (až +2 body) Najděte nějaký balíček, který Vám dovolí do Vašeho notebooku zavést interaktivní prvky, např. si vyberete v select-boxu stranu a Váš notebook zobrazí grafy pouze pro ni atp.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor(y) se staženými daty.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

In [2]:
### odtud už je to Vaše

## Postup
### Načtení dat
Načteme jmenné seznamy pro obec Olomouc.

In [132]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import csv

class DummyHeader:
    def __init__(self, str):
        self.text = str

def dummy(*strs):
    return [DummyHeader(str) for str in strs]

def download_results(url, fix_headers = lambda x: x, fix_row = lambda x: x, table_index = 0):
    results = {"headers": None, "rows": []}
    soup = BeautifulSoup(urlopen(url))
    table = soup.find_all("table")[table_index]

    if results["headers"] == None:
        results["headers"] = [h.text.lower() for h in fix_headers([h for h in table.find_all("th") if h]) if h]

    for row in table.find_all("tr"):
        fixed_row = fix_row([v for v in row.find_all("td") if v])
        results["rows"].append([v.text for v in fixed_row if v])

    return results

def save_results(file, results):
    with open(file, "w") as f:
        writer = csv.writer(f)
        writer.writerow(results["headers"])
        writer.writerows(results["rows"])
    return results

filter_supercolumns = lambda headers, filter = lambda x: True: [h for h in headers
                        if  h.text != "Kandidát"
                        and h.text != "Kandidátní listina"
                        and h.text != "Hlasy"
                        and filter(h)
                    ]

# 2002
headers2002 = dummy(
    "obvod", "číslo", "název", "poř. číslo", "příjmení, jméno", "tituly", "věk", "navrh. strana", "polit. přísl.", "abs.", "v %", "pořadí", "mandát"
)

headers_parties = dummy(
    "nazev", "hlasy abs", "hlasy %", "zastupitele abs", "zastupitele %"
)

r2002 = save_results("olomouc-2002.csv", download_results(
    "https://volby.cz/pls/kv2002/kv21111?xjazyk=CZ&xid=0&xv=11&xdz=3&xnumnuts=7102&xobec=500496&xstrana=0",
    lambda _: headers2002
))

p2002 = save_results("olomouc-parties-2002.csv", download_results(
    "https://www.volby.cz/pls/kv2002/kv1211?xjazyk=CZ&xid=0&xv=1&xdz=2&xnumnuts=7102",
    lambda _: headers_parties,
    lambda x: x,
    1
))

# 2006
headers2006_2010 = dummy(
    "obvod", "číslo", "název", "poř. číslo", "příjmení, jméno, tituly", "věk", "navrh. strana", "polit. přísl.", "abs.", "v %", "pořadí", "mandát"
)

r2006 = save_results("olomouc-2006.csv", download_results(
    "https://volby.cz/pls/kv2006/kv21111?xjazyk=CZ&xid=0&xv=11&xdz=3&xnumnuts=7102&xobec=500496&xstrana=0",
    lambda _: headers2006_2010
))

p2006 = save_results("olomouc-parties-2006.csv", download_results(
    "https://www.volby.cz/pls/kv2006/kv1111?xjazyk=CZ&xid=0&xdz=3&xnumnuts=7102&xobec=500496&xobecnaz=OLOMOUC&xstat=0&xvyber=0",
    lambda _: headers_parties,
    lambda x: x,
    1
))

# 2010
r2010 = save_results("olomouc-2010.csv", download_results(
    "https://www.volby.cz/pls/kv2010/kv21111?xjazyk=CZ&xid=0&xv=11&xdz=3&xnumnuts=7102&xobec=500496&xstrana=0",
    lambda _: headers2006_2010
))

p2010 = save_results("olomouc-parties-2010.csv", download_results(
    "https://www.volby.cz/pls/kv2006/kv1111?xjazyk=CZ&xid=0&xdz=3&xnumnuts=7102&xobec=500496&xobecnaz=OLOMOUC&xstat=0&xvyber=0",
    lambda _: headers_parties,
    lambda x: x,
    1
))

# 2014
headers2014_2018 = dummy(
    "číslo", "název", "poř. číslo", "příjmení, jméno, tituly", "věk", "navrh. strana", "polit. přísl.", "abs.", "v %", "pořadí", "mandát"
)

r2014 = save_results("olomouc-2014.csv", download_results(
    "https://www.volby.cz/pls/kv2014/kv21111?xjazyk=CZ&xid=0&xv=11&xdz=3&xnumnuts=7102&xobec=500496&xstrana=0",
    lambda _: headers2014_2018
))

p2014 = save_results("olomouc-parties-2014.csv", download_results(
    "https://www.volby.cz/pls/kv2006/kv1111?xjazyk=CZ&xid=0&xdz=3&xnumnuts=7102&xobec=500496&xobecnaz=OLOMOUC&xstat=0&xvyber=0",
    lambda _: headers_parties,
    lambda x: x,
    1
))

# 2018
r2018 = save_results("olomouc-2018.csv", download_results(
    "https://www.volby.cz/pls/kv2018/kv21111?XJAZYK=CZ&XID=0&XV=11&XDZ=3&XNUMNUTS=7102&XOBEC=500496&XSTRANA=0&xf=1",
    lambda _: headers2014_2018
))

p2018 = save_results("olomouc-parties-2018.csv", download_results(
    "https://www.volby.cz/pls/kv2006/kv1111?xjazyk=CZ&xid=0&xdz=3&xnumnuts=7102&xobec=500496&xobecnaz=OLOMOUC&xstat=0&xvyber=0",
    lambda _: headers_parties,
    lambda x: x,
    1
))


### Zpracování dat
Starší verze tabulky (2002) má oproti pozdějším trochu jiné názvy sloupců.

In [105]:
import numpy as np
import pandas as pd
import sklearn as skit
import matplotlib.pyplot as plt
import seaborn as sns

In [124]:
data2002    = pd.read_csv("olomouc-2002.csv")
parties2002 = pd.read_csv("olomouc-parties-2002.csv")

data2006    = pd.read_csv("olomouc-2006.csv")
parties2006 = pd.read_csv("olomouc-parties-2006.csv")

data2010    = pd.read_csv("olomouc-2010.csv")
parties2010 = pd.read_csv("olomouc-parties-2010.csv")

data2014    = pd.read_csv("olomouc-2014.csv")
parties2014 = pd.read_csv("olomouc-parties-2014.csv")

data2018    = pd.read_csv("olomouc-2018.csv")
parties2018 = pd.read_csv("olomouc-parties-2018.csv")

In [129]:
parties2018.head()

,nazev,hlasy abs,hlasy %,zastupitele abs,zastupitele %
0,Křesťanská a demokratická unie - Československ...,20 192,"8,35",4,"8,89"
1,Strana zelených,16 581,"6,85",1,"2,22"
2,Česká strana sociálně demokratická,59 673,"24,66",14,"31,11"
3,Komunistická strana Čech a Moravy,27 257,"11,27",5,"11,11"
4,Občanská demokratická strana,88 178,"36,44",21,"46,67"


### Grafy

In [130]:
import matplotlib
%matplotlib inline 
matplotlib.style.use('ggplot')